**Data Enigeering**

[Repositorio](https://github.com/soyHenry/PI_ML_OPS/tree/PT?tab=readme-ov-file)

**Se define autoguardado en 60 segundos**

In [3]:
autosave 60

Autosaving every 60 seconds


**Se importan las librerías necesarias para el proyecto**

In [3]:
import zipfile, gzip, json, ast
import nltk
nltk.download()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


**Se definen las funciones extraer_json y extraer_ast, las que ayudarán con la extracción de datos de los archivos .gz**


*   extraer_json hace uso de la función json.loads() para hacer la carga de datos al dataframe
*   extraer_ast hace uso de la función ast_literal_eval() para hacer la carga de datos al dataframe





In [5]:
def extraer_json(ruta):
  data = []
  with gzip.open(ruta,'rb') as archivo:
    for line in archivo:
      data.append(json.loads(line))
  df = pd.DataFrame(data)
  archivo.close()
  return df

def extraer_ast(ruta):
  data = []
  with gzip.open(ruta,'rb') as archivo:
    for line in archivo:
      data.append(ast.literal_eval(line.decode('utf-8')))
  df = pd.DataFrame(data)
  archivo.close()
  return df

**Se importan los datos a sus respectivos DataFrames**

y se hace un archivo de respaldo

In [6]:
df_juegos = extraer_json('steam_games.json.gz')
df_juegos_respaldo = df_juegos
df_reviews = extraer_ast('user_reviews.json.gz')
df_reviews_respaldo = df_reviews
df_items = extraer_ast('users_items.json.gz')
df_items_respaldo = df_items

In [31]:
df_juegos = pd.read_parquet("games.parquet")
df_items = pd.read_parquet("items.parquet")
df_reviews = pd.read_parquet("reviews.parquet")

**Se limpian los Dataframes de duplicados y registros vacíos (todos NaN)**

In [7]:
df_juegos.dropna(how="all", inplace=True)
df_reviews.dropna(how="all", inplace=True)
df_items.dropna(how="all", inplace=True)
df_juegos.drop_duplicates(subset=['title'], inplace=True)
df_reviews.drop_duplicates(subset=['user_id'], inplace=True)
df_items.drop_duplicates(subset=['user_id'], inplace=True)

**Modificaciones aplicadas para manejar errores:**

*df_juegos*

*   Datos faltantes en "publisher" fueron reemplazados por "(none)", así como datos mal ingresados, como "-"
*   Datos faltantes en "genres" fueron reemplazados por "[]"
*   Datos faltantes en "genres" fueron reemplazados por "[]"
*   Datos faltantes en "app_name", fueron reemplazados por el valor de "title"
*   Datos faltantes en "title" fueron reemplazados por el valor de "app_name"
*   Datos faltantes en "url" fueron reemplazados por ""
*   Datos incorrectos en "release_date" fueron corregidos, como "15.01.2018"
*   Datos faltantes en "release_date" fueron reemplazados por "2003-09-12", la fecha de creación de steam, además, se realizó el mismo proceso con fechas inválidas como "Soon..." y "Coming Soon"
*   Datos faltantes en "tags" fueron reemplazados por "[]"
*   Datos faltantes en "reviews_url" fueron reemplazados con "https://store.steampowered.com", la página oficial de Steam
*   Datos faltantes en "specs" fueron reemplazados por "[]"
*   Datos faltantes en "price" fueron reemplazados por "-2", mientras que "Free", "Free to play" y sus variaciones fueron  identificados con "0"
*   Datos faltantes en "developer" fueron reemplazados por "[]"
*   En caso de faltar "app_name" y "title" se elimina el registro por falta de identificación propia
*   Se convirtieron los datos de la columna id en valores numéricos
*   Se generan "id"s nuevas para los datos faltantes, continuando con la numeración máxima en el registro
*   Unión de datos de las columnas "tags" y "genres", así como ordenamiento alfabético de sus contenidos
*   Ordenamiento de specs por orden alfabético

*df_reviews*

*   Se desempaquetan los datos, ya que la columna "reviews" contenía datos de "funny", "posted", "last_edited", "item_id", "helpful", "recommend" y "review"
*   Se convirtieron los datos de la columna "funny" en sólo los valores numéricos, reemplazando por "0" cuando ningún usuario marcó el juego como "funny"
*   se reemplazaron los valores vacíos de la columna "last_edited" por "Not edited"

*df_items*

*   No fue necesario hacer manejo de datos faltantes

In [8]:
'''df_juegos'''
# Manejo de datos faltantes
df_juegos['publisher'].fillna('(none)', inplace=True)
df_juegos['publisher'].replace('-', '(none)', inplace=True)
df_juegos['publisher'].replace('---', '(none)', inplace=True)
df_juegos['genres'].fillna('[]', inplace=True)
df_juegos['app_name'].fillna(df_juegos['title'], inplace=True)
df_juegos['title'].fillna(df_juegos['app_name'], inplace=True)
df_juegos['url'].fillna('', inplace=True)
df_juegos['release_date'] = pd.to_datetime(df_juegos['release_date'], format='%d.%m.%Y', errors='coerce').dt.strftime('%Y-%m-%d')
df_juegos['release_date'].fillna('2003-09-12', inplace=True)
df_juegos['tags'].fillna('[]', inplace=True)
df_juegos['reviews_url'].fillna('https://store.steampowered.com', inplace=True)
df_juegos['specs'].fillna('[]', inplace=True)
df_juegos['price'].fillna(-2, inplace=True)
df_juegos['price'].replace({'Free': 0, 'Free To Play': -0, 'Free Demo': 0, 'Free HITMAN™ Holiday Pack': 0, 'Free Mod': 0, 'Free Movie': 0, 'Free to Play': 0, 'Free to Try': 0, 'Free to Use': 0, 'Install Now': 0, 'Install Theme': 0, 'Play Now': -2, 'Play WARMACHINE: Tactics Demo': 0, 'Play for Free!': 0, 'Play the Demo': 0, 'Starting at $449.00': -2, 'Starting at $499.00': -2, 'Third-party': -2}, inplace=True)
df_juegos['developer'].fillna('[]', inplace=True)

# Eliminación de registro en ausencia de app_name y title
df_juegos.dropna(subset=['app_name', 'title'], how='all', inplace=True)

# conversión de "id" a valores numéricos
df_juegos['id'] = pd.to_numeric(df_juegos['id'], errors='coerce')

# Generación de nuevos "id"
max_id = df_juegos['id'].max()
filas_sin_id = df_juegos[df_juegos['id'].isna()]
nuevas_id = range(int(max_id) + 1, int(max_id) + 1 + len(filas_sin_id))
df_juegos.loc[df_juegos['id'].isna(), 'id'] = nuevas_id

# Cambio de nombre de columna "id"
df_juegos.rename(columns={'id': 'item_id'}, inplace=True)


# Union y ordenamiento de genres y tags
df_juegos['genres'] = df_juegos['genres'].apply(sorted).apply(str)
df_juegos['tags'] = df_juegos['tags'].apply(sorted).apply(str)
df_juegos['tags'] = df_juegos[['genres', 'tags']].agg(' '.join, axis=1)
df_juegos.drop(columns=['genres'], inplace=True)


# Ordenamiento de specs
df_juegos['specs'] = df_juegos['specs'].apply(sorted).apply(str)







C:\Users\david\AppData\Local\Temp\ipykernel_10748\2405731985.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_juegos['publisher'].fillna('(none)', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_10748\2405731985.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [9]:
'''df_reviews'''

# Desempaquetado de columna reviews
datos = []
for indice, fila in df_reviews.iterrows():
    id = fila['user_id']
    url = fila['user_url']
    datos.extend([{'user_id': id, 'user_url': url, **review} for review in fila['reviews']])
df_reviews = pd.DataFrame(datos)

# Manejo de datos de la columna funny a valores numéricos
df_reviews['funny'] = df_reviews['funny'].replace('', '0')
df_reviews['funny'] = df_reviews['funny'].str.extract('(\d+)').astype(int)

# Manejo de datos faltantes
df_reviews['last_edited'] = df_reviews['last_edited'].replace('', 'Not edited')

In [10]:
'''df_items'''

# Desempaquetado de columna items
data = []
for indice, fila in df_items.iterrows():
    id = fila['user_id']
    url = fila['user_url']
    if 'items' in fila and isinstance(fila['items'], list):
        data.extend([{'user_id': id, 'user_url': url, **item} for item in fila['items']])
df_items = pd.DataFrame(data)


**Según lo visualizado, se harán las siguientes conversiones a los tipos de datos:**
*   `df_juegos.release_date` será cambiado de object a datetime
*   `df_juegos.early_access` será cambiado de object a booleano
*   `df_reviews.item_id` será cambiado de object a integer
*   `df_reviews.recommend` será cambiado de object a booleano
*   `df_items.item_id` será cambiado de object a integer

In [12]:
#df_juegos['release_date'] = pd.to_datetime(df_juegos['release_date'], errors='coerce')
df_juegos['early_access'] = df_juegos['early_access'].astype(bool)
df_reviews['item_id'] = df_reviews['item_id'].astype(int)
df_reviews['recommend'] = df_reviews['recommend'].astype(bool)
df_items['item_id'] = df_items['item_id'].astype(int)

**Creación de la columna "sentiment_analysis" en "df_reviews"**

In [11]:
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])

In [17]:
df_reviews.drop('review', axis=1, inplace=True)

In [13]:
df_juegos = df_juegos.drop('publisher', axis=1)
df_juegos = df_juegos.drop('title', axis=1)
df_juegos = df_juegos.drop('url', axis=1)
df_juegos = df_juegos.drop('reviews_url', axis=1)
df_juegos = df_juegos.drop('specs', axis=1)
df_juegos = df_juegos.drop('price', axis=1)
df_juegos = df_juegos.drop('early_access', axis=1)
df_juegos = df_juegos.drop('developer', axis=1)
df_items = df_items.drop('user_url', axis=1)
df_items = df_items.drop('item_name', axis=1)
df_items = df_items.drop('playtime_2weeks', axis=1)
df_reviews = df_reviews.drop('user_url', axis=1)
df_reviews = df_reviews.drop('funny', axis=1)
df_reviews = df_reviews.drop('posted', axis=1)
df_reviews = df_reviews.drop('last_edited', axis=1)
df_reviews = df_reviews.drop('helpful', axis=1)

In [19]:
df_juegos.info



<bound method DataFrame.info of                         app_name release_date  \
index                                           
88310        Lost Summoner Kitty   2003-09-12   
88311                  Ironbound   2003-09-12   
88312    Real Pool 3D - Poolians   2003-09-12   
88313                    弹炸人2222   2003-09-12   
88314              Log Challenge   2003-09-12   
...                          ...          ...   
120439              Kebab it Up!   2003-09-12   
120440            Colony On Mars   2003-09-12   
120441  LOGistICAL: South Africa   2003-09-12   
120442             Russian Roads   2003-09-12   
120443       EXIT 2 - Directions   2003-09-12   

                                                     tags  item_id  
index                                                               
88310   ['Action', 'Casual', 'Indie', 'Simulation', 'S...   761140  
88311   ['Free to Play', 'Indie', 'RPG', 'Strategy'] [...   643980  
88312   ['Casual', 'Free to Play', 'Indie', 'Simulatio

In [36]:
df_reviews



<bound method DataFrame.info of                  user_id  item_id  recommend  sentiment_analysis
0      76561197970982479     1250       True              0.8481
1      76561197970982479    22200       True              0.2263
2      76561197970982479    43110       True              0.9117
3                js41637   251610       True              0.9566
4                js41637   227300       True              0.9708
...                  ...      ...        ...                 ...
58425  76561198312638244       70       True              0.5574
58426  76561198312638244   362890       True              0.9786
58427        LydiaMorley   273110       True              0.7827
58428        LydiaMorley      730       True              0.5106
58429        LydiaMorley      440       True              0.8349

[58430 rows x 4 columns]>

In [16]:
df_items.info




<bound method DataFrame.info of                    user_id  item_id  playtime_forever
0        76561197970982479       10                 6
1        76561197970982479       20                 0
2        76561197970982479       30                 7
3        76561197970982479       40                 0
4        76561197970982479       50                 0
...                    ...      ...               ...
5094077  76561198329548331   346330                 0
5094078  76561198329548331   373330                 0
5094079  76561198329548331   388490                 3
5094080  76561198329548331   521570                 4
5094081  76561198329548331   519140                 3

[5094082 rows x 3 columns]>

In [ ]:
df_items.to_parquet("items.parquet")
df_reviews.to_parquet("reviews.parquet")
df_juegos.to_parquet("games.parquet")

In [46]:
def UserForGenre(genero):
    # Carga de datos
    df_reviews = pd.read_parquet("reviews.parquet", columns=['user_id', 'item_id'])
    df_juegos = pd.read_parquet("games.parquet", columns=['item_id', 'tags'])

    # Filtrado por género (buscar en la columna 'tags' del DataFrame juegos)
    df_genero = df_juegos[df_juegos["tags"].str.contains(genero, case=False)]
    df_genero = df_genero[df_genero["item_id"].isin(df_juegos[df_juegos["tags"].str.contains(genero, case=False)]['item_id'])]

    # Agrupación por usuario y cálculo de horas jugadas (asumiendo playtime_forever en df_items)
    df_items = pd.read_parquet("items.parquet", columns=['user_id', 'item_id', 'playtime_forever'])
    df_items['playtime_forever'] = df_items['playtime_forever'].astype('float32')
    df_horas = df_genero.merge(df_items, on='item_id', how='left').groupby('user_id')['playtime_forever'].sum()

    # Usuario con más horas jugadas
    usuario_max = df_horas.idxmax()

    return {"Usuario con más horas jugadas para Género " + genero: usuario_max, "Horas jugadas": df_horas.to_dict()}


In [47]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas jugadas': {'--000--': 140410.0,
  '--ace--': 69682.0,
  '--ionex--': 38534.0,
  '-2SV-vuLB-Kg': 43379.0,
  '-404PageNotFound-': 197929.0,
  '-AnimeIsMyThing-': 244002.0,
  '-Azsael-': 258241.0,
  '-Beave-': 35719.0,
  '-Encore-': 26442.0,
  '-GM-Dragon': 97161.0,
  '-I_AM_EPIC-': 19831.0,
  '-JT': 85152.0,
  '-Kenny': 170458.0,
  '-KillZone-': 96956.0,
  '-Mad-': 165627.0,
  '-Naughty-': 149199.0,
  '-PRoSlayeR-': 217163.0,
  '-SEVEN-': 265348.0,
  '-SatansLittleHelper-': 121540.0,
  '-Thyme-': 152592.0,
  '-Ultrix': 62568.0,
  '-Uplink-': 157017.0,
  '-Zovix-': 47775.0,
  '-_PussyDestroyer_-': 61457.0,
  '-_skIzZ_-': 35995.0,
  '-fastrvrs-': 3763.0,
  '-iBubble': 28248.0,
  '-kainey9777': 43990.0,
  '-xKyox-': 127187.0,
  '0-3-0': 12784.0,
  '00000000000000000001227': 96216.0,
  '00000003010': 15336.0,
  '0000FF': 75429.0,
  '000Infinity000': 1708.0,
  '001000111000111000010': 29292.0,
  '001002130882': 27246.0,
 